In [2]:
import warnings
import os
warnings.filterwarnings('ignore')
os.environ['PYTHONWARNINGS'] = 'ignore'

import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, r2_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from scipy.stats import uniform, randint

# ---------- 데이터 로드 ----------
train = pd.read_csv('./data/1.titanic_train.csv')
test = pd.read_csv('./data/2.titanic_test.csv')
submission = pd.read_csv('./data/3.titanic_submission.csv')
test_passenger_id = test['PassengerId']

# ---------- [Feature Engineering/데이터 가공] ----------
for df in [train, test]:
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['FamilyType'] = pd.cut(df['FamilySize'], bins=[0, 1, 4, 20], labels=['Alone', 'Small', 'Large'])
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
def extract_title(name):
    match = re.search(r',\s*([^.]*)\.', name)
    if match:
        return match.group(1).strip()
    return 'Unknown'
for df in [train, test]:
    df['Title'] = df['Name'].apply(extract_title)
    df['Title'] = df['Title'].replace(
        ['Lady', 'Countess','Capt','Col','Don','Dr','Major','Rev','Sir','Jonkheer','Dona'], 'Rare'
    )
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')
for df in [train, test]:
    df['Age*Pclass'] = df['Age'] * df['Pclass']
    df['Sex*Pclass'] = df.apply(lambda row: 0 if row['Sex']=='male' else 1, axis=1) * df['Pclass']
for df in [train, test]:
    df['FarePerPerson'] = df['Fare'] / (df['FamilySize'])
    df['Fare_log'] = np.log1p(df['Fare'])
for df in [train, test]:
    df['Deck'] = df['Cabin'].fillna('U').map(lambda x: x[0])
for df in [train, test]:
    for title in df['Title'].unique():
        med = df.loc[df['Title']==title, 'Age'].median()
        df.loc[(df['Title']==title) & (df['Age'].isnull()), 'Age'] = med
for df in [train, test]:
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
for df in [test]:
    for pclass in df['Pclass'].unique():
        med = df.loc[df['Pclass']==pclass, 'Fare'].median()
        df.loc[(df['Pclass']==pclass) & (df['Fare'].isnull()), 'Fare'] = med

drop_cols = ['PassengerId', 'Name', 'Ticket', 'Cabin']
train = train.drop(drop_cols, axis=1)
test = test.drop(drop_cols, axis=1)

numerical_features = train.select_dtypes(include=np.number).columns.tolist()
if 'Survived' in numerical_features: numerical_features.remove('Survived')
categorical_features = train.select_dtypes(exclude=np.number).columns.tolist()

numerical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', numerical_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
])

X = train.drop('Survived', axis=1)
y = train['Survived']
X_processed = preprocessor.fit_transform(X)
X_test_processed = preprocessor.transform(test)

X_train, X_val, y_train, y_val = train_test_split(
    X_processed, y, test_size=0.2, random_state=42, stratify=y
)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# ----------- XGBoost 최적화 -----------
xgb_base = XGBClassifier(random_state=42, eval_metric='logloss', tree_method='hist')
param_dist_xgb = {
    'n_estimators': randint(50, 500),
    'max_depth': randint(2, 10),
    'learning_rate': uniform(0.01, 0.1),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5),
    'gamma': uniform(0, 2),
    'reg_alpha': uniform(0, 2),
    'reg_lambda': uniform(0, 2)
}
rand_search_xgb = RandomizedSearchCV(
    xgb_base, param_distributions=param_dist_xgb, n_iter=10, cv=kfold, scoring='accuracy',
    verbose=0, n_jobs=-1, random_state=42
)
rand_search_xgb.fit(X_train, y_train)
best_xgb_params = rand_search_xgb.best_params_

# ----------- LightGBM 최적화 -----------
lgbm_base = LGBMClassifier(random_state=42, verbosity=-1)
param_dist_lgbm = {
    'n_estimators': randint(50, 500),
    'max_depth': randint(2, 10),
    'learning_rate': uniform(0.01, 0.1),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5),
    'reg_alpha': uniform(0, 2),
    'reg_lambda': uniform(0, 2)
}
rand_search_lgbm = RandomizedSearchCV(
    lgbm_base, param_distributions=param_dist_lgbm, n_iter=10, cv=kfold, scoring='accuracy',
    verbose=0, n_jobs=-1, random_state=42
)
rand_search_lgbm.fit(X_train, y_train)
best_lgbm_params = rand_search_lgbm.best_params_
best_lgbm_params['verbosity'] = -1

# ----------- RandomForest -----------
rfc_base = RandomForestClassifier(random_state=42)
param_grid_rfc = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 7, 12, None],
    'min_samples_split': [2, 4, 8]
}
grid_search_rfc = GridSearchCV(
    rfc_base, param_grid_rfc, cv=kfold, scoring='accuracy',
    verbose=0, n_jobs=-1
)
grid_search_rfc.fit(X_train, y_train)
best_rfc_params = grid_search_rfc.best_params_

# ----------- CatBoost -----------
catboost_base = CatBoostClassifier(random_state=42, verbose=0)
param_dist_cat = {
    'iterations': randint(60, 200),
    'depth': randint(3, 9),
    'learning_rate': uniform(0.03, 0.1),
    'l2_leaf_reg': uniform(0.5, 5)
}
rand_search_cat = RandomizedSearchCV(
    catboost_base, param_distributions=param_dist_cat, n_iter=8, cv=kfold, scoring='accuracy',
    verbose=0, n_jobs=-1, random_state=42
)
rand_search_cat.fit(X_train, y_train)
best_cat_params = rand_search_cat.best_params_
best_cat_params['verbose'] = 0

# ----------- 모델 객체 생성 (최적 파라미터) -----------
xgb_final = XGBClassifier(**best_xgb_params, random_state=42, eval_metric='logloss', tree_method='hist')
lgbm_final = LGBMClassifier(**best_lgbm_params, random_state=42)
rfc_final = RandomForestClassifier(**best_rfc_params, random_state=42)
cat_final = CatBoostClassifier(**best_cat_params, random_state=42)

# ----------- 최적 점수 및 파라미터 표시 -----------
print("=== 하이퍼파라미터 최적화 결과 ===")
model_results = {
    "XGBoost": (rand_search_xgb.best_score_, rand_search_xgb.best_params_),
    "LightGBM": (rand_search_lgbm.best_score_, rand_search_lgbm.best_params_),
    "RandomForest": (grid_search_rfc.best_score_, grid_search_rfc.best_params_),
    "CatBoost": (rand_search_cat.best_score_, rand_search_cat.best_params_)
}
for name, (score, params) in model_results.items():
    print(f"{name:12} | score={score:.4f} | params={params}")

# 어떤 모델이 최고 성능인지 출력
best_model = max(model_results.items(), key=lambda x: x[1][0])
print(f"\n>>> 가장 성능이 좋았던 모델: {best_model[0]} (score={best_model[1][0]:.4f}) ")
print(f"    최적 파라미터: {best_model[1][1]}\n")

# ----------- 스태킹 앙상블 -----------
stacking = StackingClassifier(
    estimators=[
        ('xgb', xgb_final),
        ('lgbm', lgbm_final),
        ('rf', rfc_final),
        ('cat', cat_final)
    ],
    final_estimator=LogisticRegression(random_state=42, max_iter=1000),
    cv=kfold,
    n_jobs=-1,
    passthrough=False
)
stacking.fit(X_train, y_train)

# ----------- 학습 데이터 평가 -----------
y_pred_train = stacking.predict(X_train)
y_pred_train_proba = stacking.predict_proba(X_train)
print("\n=== [Stacking 앙상블: 학습 데이터] ===")
print(classification_report(y_train, y_pred_train))
print("R2  :", r2_score(y_train, y_pred_train))
print("MAE :", mean_absolute_error(y_train, y_pred_train))
print("MSE :", mean_squared_error(y_train, y_pred_train))
print("RMSE:", np.sqrt(mean_squared_error(y_train, y_pred_train)))
print("Log Loss:", log_loss(y_train, y_pred_train_proba))

# ----------- 검증 데이터 평가 -----------
y_pred_val_proba = stacking.predict_proba(X_val)
y_pred_val = stacking.predict(X_val)
print("\n=== [Stacking 앙상블: 검증 데이터] ===")
print(classification_report(y_val, y_pred_val))
print("R2  :", r2_score(y_val, y_pred_val))
print("MAE :", mean_absolute_error(y_val, y_pred_val))
print("MSE :", mean_squared_error(y_val, y_pred_val))
print("RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_val)))
print("Log Loss:", log_loss(y_val, y_pred_val_proba))

# ----------- 전체 데이터로 재학습 -----------
stacking.fit(X_processed, y)

# ----------- 최종 제출 예측 -----------
y_test_pred = stacking.predict(X_test_processed)
submission['Survived'] = y_test_pred
submission['PassengerId'] = test_passenger_id
submission.to_csv('titanic_stacking_submission.csv', index=False)
print("\n제출 파일 저장 완료!(titanic_stacking_submission.csv)")


=== 하이퍼파라미터 최적화 결과 ===
XGBoost      | score=0.8331 | params={'colsample_bytree': 0.5924272277627636, 'gamma': 1.9391692555291171, 'learning_rate': 0.08751328233611146, 'max_depth': 3, 'n_estimators': 251, 'reg_alpha': 1.7896547008552977, 'reg_lambda': 1.1957999576221703, 'subsample': 0.9609371175115584}
LightGBM     | score=0.8231 | params={'colsample_bytree': 0.7838501639099957, 'learning_rate': 0.01313132924555586, 'max_depth': 3, 'n_estimators': 267, 'reg_alpha': 0.8995082667395313, 'reg_lambda': 0.7903004720036289, 'subsample': 0.9633294328968971, 'verbosity': -1}
RandomForest | score=0.8331 | params={'max_depth': 12, 'min_samples_split': 8, 'n_estimators': 100}
CatBoost     | score=0.8264 | params={'depth': 5, 'iterations': 147, 'l2_leaf_reg': 2.168543055695109, 'learning_rate': 0.044286681792194076, 'verbose': 0}

>>> ⭐ 가장 성능이 좋았던 모델: XGBoost (score=0.8331) 
    최적 파라미터: {'colsample_bytree': 0.5924272277627636, 'gamma': 1.9391692555291171, 'learning_rate': 0.08751328233611146, 'm